In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'>
import numpy as np
from sklearn.model_selection import train_test_split
import collections
import sys
from mlxtend.evaluate import confusion_matrix
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from matplotlib import cm as cmap
import plotly.express as px
import psycopg2
from psycopg2.extras import execute_values

import warnings
warnings.filterwarnings("ignore")
import mlflow
from sklearn.metrics import accuracy_score, recall_score, f1_score, average_precision_score
from urllib.parse import urlparse

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



# 1. Best models

In [3]:
def get_experiment_id(base_name,mlflow):
    n = 0
    experiment_id = 0
    while True:
        try:
            name = mlflow.get_experiment(n).name
        except:
            break
        print(name)
        if base_name == name:
            experiment_id = n
            break
        else:
            n = n + 1
    return experiment_id

## Show best models URI

In [7]:
mlflow.set_tracking_uri("http://nginx:80")
    
experiment_name     = 'stroke_demo_airflow'
experiment_id = get_experiment_id(experiment_name,mlflow)
print(experiment_name,experiment_id)

# best artifact uri
runs_metadata = mlflow.search_runs([experiment_id])
best_artifact_uri = runs_metadata.sort_values(by='metrics.accuracy', ascending=False)['artifact_uri']
best_artifact_uri

Default
stroke_demo
stroke_demo_airflow
stroke_demo_airflow 2


5     s3://mlflow-bucket/mlflow/2/f75705cb01384f4c8c...
4     s3://mlflow-bucket/mlflow/2/98a7d9fe9c9c4d6b80...
1     s3://mlflow-bucket/mlflow/2/44d0ef11caae4384bb...
3     s3://mlflow-bucket/mlflow/2/6a65fd92dbd84f9fa8...
2     s3://mlflow-bucket/mlflow/2/75a8af52df0a48df88...
0     s3://mlflow-bucket/mlflow/2/30fc2c107c9c4acb92...
11    s3://mlflow-bucket/mlflow/2/f99862d9229446c882...
15    s3://mlflow-bucket/mlflow/2/62d96303dc094588b0...
14    s3://mlflow-bucket/mlflow/2/78b471c6a42447af98...
13    s3://mlflow-bucket/mlflow/2/18c9f475c25c44c59a...
12    s3://mlflow-bucket/mlflow/2/2174a3b676c346208b...
8     s3://mlflow-bucket/mlflow/2/457712eddb26411baf...
10    s3://mlflow-bucket/mlflow/2/576f27d21fe646f09f...
9     s3://mlflow-bucket/mlflow/2/ccc38096a89a45a3bc...
7     s3://mlflow-bucket/mlflow/2/8b193c76dfa5454b85...
6     s3://mlflow-bucket/mlflow/2/f2258f6dee7e4b4ca4...
16    s3://mlflow-bucket/mlflow/2/8d89da5f79ff45cf9e...
17    s3://mlflow-bucket/mlflow/2/6d2f333143c64d

In [10]:
## Show best models info

In [9]:
results = mlflow.search_runs([experiment_id])
results

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,metrics.recall,metrics.f1,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.log-model.history,tags.mlflow.source.name
0,30fc2c107c9c4acb92d499ef81a5d203,2,FINISHED,s3://mlflow-bucket/mlflow/2/30fc2c107c9c4acb92...,2021-05-07 05:10:06.671000+00:00,2021-05-07 05:10:55.030000+00:00,0.750000,0.750000,0.722222,LOCAL,airflow,"[{""run_id"": ""30fc2c107c9c4acb92d499ef81a5d203""...",/usr/local/bin/airflow
1,44d0ef11caae4384bbc76f9425d0fdef,2,FINISHED,s3://mlflow-bucket/mlflow/2/44d0ef11caae4384bb...,2021-05-07 04:10:10.068000+00:00,2021-05-07 04:10:49.154000+00:00,0.882353,0.882353,0.869406,LOCAL,airflow,"[{""run_id"": ""44d0ef11caae4384bbc76f9425d0fdef""...",/usr/local/bin/airflow
2,75a8af52df0a48df88a4aee71a6dbb1e,2,FINISHED,s3://mlflow-bucket/mlflow/2/75a8af52df0a48df88...,2021-05-07 03:10:05.515000+00:00,2021-05-07 03:10:49.425000+00:00,0.781250,0.781250,0.755952,LOCAL,airflow,"[{""run_id"": ""75a8af52df0a48df88a4aee71a6dbb1e""...",/usr/local/bin/airflow
3,6a65fd92dbd84f9fa81d64e93370cbdc,2,FINISHED,s3://mlflow-bucket/mlflow/2/6a65fd92dbd84f9fa8...,2021-05-07 02:10:08.289000+00:00,2021-05-07 02:10:51.527000+00:00,0.833333,0.833333,0.856209,LOCAL,airflow,"[{""run_id"": ""6a65fd92dbd84f9fa81d64e93370cbdc""...",/usr/local/bin/airflow
4,98a7d9fe9c9c4d6b80892185cab8af90,2,FINISHED,s3://mlflow-bucket/mlflow/2/98a7d9fe9c9c4d6b80...,2021-05-07 01:10:05.557000+00:00,2021-05-07 01:10:47.913000+00:00,0.892857,0.892857,0.880495,LOCAL,airflow,"[{""run_id"": ""98a7d9fe9c9c4d6b80892185cab8af90""...",/usr/local/bin/airflow
5,f75705cb01384f4c8cbfb24e5c6ac2eb,2,FINISHED,s3://mlflow-bucket/mlflow/2/f75705cb01384f4c8c...,2021-05-07 00:10:09.941000+00:00,2021-05-07 00:10:40.412000+00:00,0.980769,0.980769,0.971247,LOCAL,airflow,"[{""run_id"": ""f75705cb01384f4c8cbfb24e5c6ac2eb""...",/usr/local/bin/airflow
6,f2258f6dee7e4b4ca4404102046ddc53,2,FINISHED,s3://mlflow-bucket/mlflow/2/f2258f6dee7e4b4ca4...,2021-05-06 23:10:08.538000+00:00,2021-05-06 23:10:24.197000+00:00,0.000000,0.000000,0.000000,LOCAL,airflow,"[{""run_id"": ""f2258f6dee7e4b4ca4404102046ddc53""...",/usr/local/bin/airflow
7,8b193c76dfa5454b85d34bb2538c9cc7,2,FINISHED,s3://mlflow-bucket/mlflow/2/8b193c76dfa5454b85...,2021-05-06 22:10:05.957000+00:00,2021-05-06 22:10:23.249000+00:00,0.000000,0.000000,0.000000,LOCAL,airflow,"[{""run_id"": ""8b193c76dfa5454b85d34bb2538c9cc7""...",/usr/local/bin/airflow
8,457712eddb26411baf4cdc7a1740c6f4,2,FINISHED,s3://mlflow-bucket/mlflow/2/457712eddb26411baf...,2021-05-06 21:10:08.332000+00:00,2021-05-06 21:10:21.150000+00:00,0.000000,0.000000,0.000000,LOCAL,airflow,"[{""run_id"": ""457712eddb26411baf4cdc7a1740c6f4""...",/usr/local/bin/airflow
9,ccc38096a89a45a3bc952f031d52897b,2,FINISHED,s3://mlflow-bucket/mlflow/2/ccc38096a89a45a3bc...,2021-05-06 20:10:09.871000+00:00,2021-05-06 20:10:24.279000+00:00,0.000000,0.000000,0.000000,LOCAL,airflow,"[{""run_id"": ""ccc38096a89a45a3bc952f031d52897b""...",/usr/local/bin/airflow


## Metrics over time

In [42]:
fig = px.scatter(title="Metrics over time")
fig.add_scatter(x=results["end_time"], y=results["metrics.f1"],name="f1")
fig.add_scatter(x=results["end_time"], y=results["metrics.accuracy"],name="accuracy")
fig.add_scatter(x=results["end_time"], y=results["metrics.recall"],name="recall")
fig.show()

## Metrics over versions

In [53]:
results_odered = results.sort_values("end_time").reset_index()

In [54]:
fig = px.scatter(title="Metrics over versions")
fig.add_scatter(x=results_odered.index, y=results_odered["metrics.f1"],name="f1")
fig.add_scatter(x=results_odered.index, y=results_odered["metrics.accuracy"],name="accuracy")
fig.add_scatter(x=results_odered.index, y=results_odered["metrics.recall"],name="recall")
fig.show()